In [1]:
pip install pygame

You should consider upgrading via the 'c:\venv\ilumpy\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import pygame
import random
import math

pygame.init()

# Basic parameters of the screen
WIDTH, HEIGHT = 900, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong with Random Positions and Momentum Conservation")

clock = pygame.time.Clock()
FPS = 60

# RGB values of standard colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)

# Ball class
class Ball:
    def __init__(self, posx, posy, diameter, speed, color, weight):
        self.posx = posx
        self.posy = posy
        self.diameter = diameter
        self.radius = diameter / 2
        self.speed = speed
        self.color = color
        self.weight = weight
        self.xVel = self.speed * random.choice([-1, 1])
        self.yVel = self.speed * random.choice([-1, 1])
        self.ball = pygame.draw.circle(screen, self.color, (int(self.posx), int(self.posy)), self.radius)

    def display(self):
        self.ball = pygame.draw.circle(screen, self.color, (int(self.posx), int(self.posy)), self.radius)

    def update(self):
        self.posx += self.xVel
        self.posy += self.yVel
        if self.posy <= self.radius or self.posy >= HEIGHT - self.radius:
            self.yVel *= -1
        if self.posx <= self.radius or self.posx >= WIDTH - self.radius:
            self.xVel *= -1

    def check_collision(self, other_ball):
        dx = other_ball.posx - self.posx
        dy = other_ball.posy - self.posy
        distance = math.sqrt(dx**2 + dy**2)

        if distance <= self.radius + other_ball.radius:
            overlap = (self.radius + other_ball.radius) - distance
            dx /= distance
            dy /= distance

            self.posx -= overlap * dx * 0.5
            self.posy -= overlap * dy * 0.5
            other_ball.posx += overlap * dx * 0.5
            other_ball.posy += overlap * dy * 0.5

            total_weight = self.weight + other_ball.weight

            self_momentum = self.weight * math.sqrt(self.xVel**2 + self.yVel**2)
            other_momentum = other_ball.weight * math.sqrt(other_ball.xVel**2 + other_ball.yVel**2)

            self_speed = self_momentum / self.weight
            other_speed = other_momentum / other_ball.weight

            self_angle = math.atan2(self.yVel, self.xVel)
            other_angle = math.atan2(other_ball.yVel, other_ball.xVel)

            self_new_xvel = self_speed * math.cos(self_angle)
            self_new_yvel = self_speed * math.sin(self_angle)
            other_new_xvel = other_speed * math.cos(other_angle)
            other_new_yvel = other_speed * math.sin(other_angle)

            self.xVel = (self_new_xvel * (self.weight - other_ball.weight) + 2 * other_ball.weight * other_new_xvel) / total_weight
            self.yVel = (self_new_yvel * (self.weight - other_ball.weight) + 2 * other_ball.weight * other_new_yvel) / total_weight
            other_ball.xVel = (other_new_xvel * (other_ball.weight - self.weight) + 2 * self.weight * self_new_xvel) / total_weight
            other_ball.yVel = (other_new_yvel * (other_ball.weight - self.weight) + 2 * self.weight * self_new_yvel) / total_weight

    def getRect(self):
        return pygame.Rect(self.posx - self.radius, self.posy - self.radius, self.diameter, self.diameter)

def main(num_balls_white, num_balls_yellow, weight_white, weight_yellow, diameter_white, diameter_yellow):
    running = True
    balls = [Ball(random.randint(0 + diameter_white, WIDTH - diameter_white), random.randint(0 + diameter_white, HEIGHT - diameter_white), diameter_white, 3, WHITE, weight_white) for _ in range(num_balls_white)]
    yellow_balls = [Ball(random.randint(0 + diameter_yellow, WIDTH - diameter_yellow), random.randint(0 + diameter_yellow, HEIGHT - diameter_yellow), diameter_yellow, 3, YELLOW, weight_yellow) for _ in range(num_balls_yellow)]

    while running:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        all_balls = balls + yellow_balls

        for i in range(len(all_balls)):
            for j in range(i + 1, len(all_balls)):
                all_balls[i].check_collision(all_balls[j])

        for ball in all_balls:
            ball.update()
            ball.display()

        pygame.display.update()
        clock.tick(FPS)

    pygame.quit()

if __name__ == "__main__":
    num_balls_white = 2  # Change this to the desired number of white balls
    num_balls_yellow = 2  # Change this to the desired number of yellow balls
    weight_white = 1  # Change this to the desired weight for white balls
    weight_yellow = 4  # Change this to the desired weight for yellow balls
    diameter_white = 10  # Change this to the desired diameter for white balls
    diameter_yellow = 22  # Change this to the desired diameter for yellow balls
    main(num_balls_white, num_balls_yellow, weight_white, weight_yellow, diameter_white, diameter_yellow)